### Importing the libraries

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,Flatten,Dense,Dropout

In [2]:
# path to your dataset
path = 'data'

In [3]:
folders = os.listdir(path)
folders

['without_mask', 'with_mask']

### Loading the images and labeling them

In [4]:
#initializations
data = []
#looping through all the folders
for folder in folders: 
    new_path = os.path.join(path,folder)
    total = len(os.listdir(new_path))
    label = folders.index(folder)
    print('folder_name = {} and no of images = {}'.format(folder,total))
    for img in os.listdir(new_path):#loop through all the images in that folder
    #read the images one by one
        try:
            img = cv2.imread( os.path.join(new_path,img),0) #grayscale
            img = cv2.resize(img,(100,100)) #check here once
            data.append([img,label])
        except Exception as e:
            print('Exception ',e)

folder_name = without_mask and no of images = 686
folder_name = with_mask and no of images = 690


### Shuffling the images randomly
* As we have loaded the images sequentially, it is better to shuffle the images randomly.

In [5]:
#shuffle the data in place
import random
random.shuffle(data)

In [6]:
#seperating images are labels after shuffling
images = []
labels = []
for image,label in data:
    images.append(image)
    labels.append(label)

In [7]:
#normalizing the images 
images = np.array(images)/255
labels = np.array(labels)

In [8]:
images.shape

(1376, 100, 100)

In [9]:
#converting to 4D 
images = np.expand_dims(images,axis = 3)

In [10]:
images.shape

(1376, 100, 100, 1)

In [11]:
# seperating train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(images,labels,test_size = 0.1,random_state = 0)

### Model Building

In [12]:
#initializing the model
model=Sequential()

#The first CNN layer followed by Relu and MaxPooling layers
model.add(Conv2D(200,(3,3),activation = 'relu',input_shape=images.shape[1:]))
model.add(MaxPooling2D())

#The second convolution layer followed by Relu and MaxPooling layers
model.add(Conv2D(100,(3,3),activation = 'relu'))
model.add(MaxPooling2D())

#Flattening 
model.add(Flatten())

#Adding dropout to avoid overfitting
model.add(Dropout(0.5))

#Dense Layer of 50 neurons
model.add(Dense(50,activation='relu'))

#Output layer
model.add(Dense(1,activation='sigmoid'))

In [13]:
#summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 52900)             0         
_________________________________________________________________
dropout (Dropout)            (None, 52900)             0         
_________________________________________________________________
dense (Dense)                (None, 50)                2

In [14]:
model.compile(optimizer='adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

### Model Training and Evaluation

In [15]:
# BATCH_SIZE = 32
EPOCHS = 10
history=model.fit(X_train,y_train,epochs=EPOCHS, validation_data = (X_test,y_test)) #,callbacks=[checkpoint]

Epoch 1/10
39/39 [==============================] - 56s 1s/step - loss: 0.6773 - accuracy: 0.5541 - val_loss: 0.4926 - val_accuracy: 0.8188
Epoch 2/10
39/39 [==============================] - 56s 1s/step - loss: 0.4693 - accuracy: 0.7843 - val_loss: 0.2584 - val_accuracy: 0.9130
Epoch 3/10
39/39 [==============================] - 55s 1s/step - loss: 0.3069 - accuracy: 0.8845 - val_loss: 0.1789 - val_accuracy: 0.9565
Epoch 4/10
39/39 [==============================] - 55s 1s/step - loss: 0.2302 - accuracy: 0.9168 - val_loss: 0.1405 - val_accuracy: 0.9493
Epoch 5/10
39/39 [==============================] - 55s 1s/step - loss: 0.1351 - accuracy: 0.9548 - val_loss: 0.0815 - val_accuracy: 0.9855
Epoch 6/10
39/39 [==============================] - 54s 1s/step - loss: 0.0898 - accuracy: 0.9661 - val_loss: 0.0606 - val_accuracy: 0.9710
Epoch 7/10
39/39 [==============================] - 55s 1s/step - loss: 0.0645 - accuracy: 0.9782 - val_loss: 0.2034 - val_accuracy: 0.9203
Epoch 8/10
39/39 [==

### Save the model

In [16]:
model_name = 'model.h5'
path = 'C:/Users/Rudresh Savalgi/Desktop/Face-Mask-Detection' + '/' + model_name
#save the model
model.save('C:/Users/Rudresh Savalgi/Desktop/Face-Mask-Detection/model.h5')